In [1]:
%load_ext autoreload
%load_ext autotime

time: 318 µs (started: 2023-03-08 08:38:50 -08:00)


In [2]:
%run init.ipynb

/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': None, 'dashboard_address': ':8787', 'security': None}}
time: 2.38 s (started: 2023-03-08 08:38:50 -08:00)


In [5]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 8.39 ms (started: 2023-03-08 08:39:16 -08:00)


In [6]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

time: 1.36 ms (started: 2023-03-08 08:39:24 -08:00)


In [7]:
client

<Client: 'tcp://192.168.130.70:33329' processes=64 threads=64, memory=2.91 TiB>

time: 91.6 ms (started: 2023-03-08 08:39:32 -08:00)


In [8]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 1.05 ms (started: 2023-03-08 08:39:34 -08:00)


In [9]:
DELTA = 0.0001

time: 663 µs (started: 2023-03-08 08:39:34 -08:00)


In [10]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

PermissionError: [Errno 13] Failed to open local file '/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/0_0.parquet'. Detail: [errno 13] Permission denied

time: 1.69 s (started: 2023-03-08 08:39:36 -08:00)


In [ ]:
ddf.head()

In [9]:
ddf.index.count().compute()

715248240

time: 12.7 s (started: 2023-03-07 18:05:48 -08:00)


In [10]:
ddf['tend'].max().compute()

3256.26416015625

time: 1.07 s (started: 2023-03-07 18:06:01 -08:00)


In [11]:
ddf['file_id'].unique().count().compute()

21267734

time: 6.85 s (started: 2023-03-07 18:06:02 -08:00)


In [12]:
ddf['proc_id'].unique().count().compute()

3990

time: 1.02 s (started: 2023-03-07 18:06:09 -08:00)


In [13]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmid_min, tmid_max

(0, 32562641674)

time: 2.95 ms (started: 2023-03-07 18:06:10 -08:00)


In [14]:
trange = np.arange(tmid_min, tmid_max, 1e7)
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 13.3 ms (started: 2023-03-07 18:06:10 -08:00)


In [15]:
max_io_time = ddf.groupby(['proc_id']).sum()['duration'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 21.7 ms (started: 2023-03-07 18:06:10 -08:00)


In [16]:
VIEW_TYPES = ['trange', 'file_id', 'proc_id']

time: 537 µs (started: 2023-03-07 18:06:10 -08:00)


In [17]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(ddf: dd.DataFrame, delta: float, max_io_time: float, metric='duration'):
    def set_delta(df: pd.DataFrame):
        df['duration_csp'] = df['duration_sum'].cumsum() / max_io_time
        df['duration_delta'] = df['duration_csp'].diff().fillna(df['duration_csp'])
        df['duration_score'] = np.digitize(df['duration_delta'], bins=DELTA_BINS, right=True)
        df['duration_cut'] = np.choose(df['duration_score'] - 1, choices=DELTA_BINS, mode='clip')
        return df
    ddf = ddf.map_partitions(set_delta)
    return ddf[ddf['duration_delta'] > delta]

time: 4.31 ms (started: 2023-03-07 18:06:10 -08:00)


In [18]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[ddf['cat'] == 0] \
    .map_partitions(lambda df: df.assign(proc_name=(
        df['app']
        .str.cat(df['hostname'], sep='-')
        .str.cat(df['rank'].astype(str), sep='-')
        .str.lstrip('-')
        .str.rstrip('-')
    ))) \
    .map_partitions(lambda df: df.assign(file_name=df['filename'])) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first']
#         'func_id': [unique_flatten()]
    }) \
    .reset_index() \
    .persist()

main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]

main_view


,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name_first,proc_name_first
npartitions=1,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float32,int64,int64,int64,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...


time: 360 ms (started: 2023-03-07 18:06:10 -08:00)


In [19]:
# wait(main_view)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('reset_index-a59a31f475787a41d2b4505c67ea5637', 0)>}, not_done=set())

time: 4min 43s (started: 2023-03-07 17:57:34 -08:00)


In [20]:
# main_view.head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name_first,proc_name_first
0,1,113104469769545699,109922137388615755,3,0,0.013678,511,0,0,0,UNKNOWN,individuals-lassen338-0
1,1,113104469769545699,109922206108092491,3,0,0.000231,511,0,0,0,UNKNOWN,individuals-lassen338-0
2,1,113104469769545699,109922223287961675,3,0,0.000240,511,0,0,0,UNKNOWN,individuals-lassen338-0
3,1,113104469769545699,109922300597373003,3,0,0.000238,511,0,0,0,UNKNOWN,individuals-lassen338-0
4,1,113104469769545699,109922322072209483,3,0,0.000233,511,0,0,0,UNKNOWN,individuals-lassen338-0


time: 26.4 s (started: 2023-03-07 18:02:18 -08:00)


In [19]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
#main_view['metadata_count'] = main_view['metadata_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)


final_view = main_view \
    .groupby(['trange', 'file_id', 'proc_id']) \
    .agg({
        'duration_sum': sum,
        'index_count': sum,
        'size_sum': sum,
        'read_count': sum,
        'write_count': sum,
        'metadata_count': sum,
        'read_size': sum,
        'write_size': sum,
        'metadata_size': sum,
        'read_time': sum,
        'write_time': sum,
        'metadata_time': sum,
        'sequential_count': sum,
        'random_count': sum,
        'sequential_size': sum,
        'random_size': sum,
        'sequential_time': sum,
        'random_time': sum,
        'data_count': sum,
        'data_size': sum,
        'data_time': sum,
#         'workload_type': sum,
        'size_min': min,
        'size_max': max,
        'file_name_first': 'first',
        'proc_name_first': 'first'
    }) \
    .reset_index() 
mask_hostname = int('0000000000000000111111111111111100000000000000000000000000000000', 2)

final_view['hostname_id'] = final_view['proc_id'] & mask_hostname
final_view = final_view.persist()


time: 350 ms (started: 2023-03-07 18:06:10 -08:00)


In [20]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: error, key: ('assign-2f80d295e63e457276aa1bd02d38663b', 0)>}, not_done=set())

time: 20min 20s (started: 2023-03-07 18:06:11 -08:00)


In [ ]:
final_view.head()

In [ ]:
del main_view

In [ ]:
final_view.head()

In [ ]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(view_type: str, ddf: dd.DataFrame, delta: float, metric: str, max_io_time: dd.core.Scalar):
    metric_col, csp_col, delta_col, score_col, cut_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
        f"{metric}_score",
        f"{metric}_cut"
    )

    def set_delta(df: pd.DataFrame):
        df[csp_col] = df[metric_col].cumsum() / max_io_time
        df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
        df[score_col] = np.digitize(df[delta_col], bins=DELTA_BINS, right=True)
        df[cut_col] = np.choose(df[score_col] - 1, choices=DELTA_BINS, mode='clip')
        return df

    index = pd.MultiIndex(levels=[[1]], codes=[[]], names=[view_type])
    meta = dd.utils.make_meta(([
        (metric_col, np.float64),
        (csp_col, np.float64),
        (delta_col, np.float64),
        (score_col, np.int8),
        (cut_col, np.float16),
    ]), index=index)

#     ddf = ddf.map_partitions(set_delta, meta=meta)
    ddf = ddf.map_partitions(set_delta)

    return ddf[ddf[delta_col] > delta]


In [ ]:
max_io_time = final_view.groupby(['proc_id']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

In [ ]:
import itertools as it

def _set_metric_percentages(ddf: dd.DataFrame, max_io_time: dd.core.Scalar, metric: str):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    ddf[pero_col] = ddf[metric_col] / max_io_time
    ddf[perr_col] = ddf[metric_col] / ddf[metric_col].sum()
    return ddf

def _compute_expanded_view(
    parent_view: dd.DataFrame,
    view_type: str,
    cols: tuple,
    metric: str,
    delta: float,
    max_io_time: dd.core.Scalar
):
    # Get column names
    metric_col, score_col, cut_col = cols
    # Check view type
    if view_type is not 'proc_id':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_id']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Filter view
    filtered_view = filter_delta(
        view_type=view_type,
        ddf=group_view,
        delta=delta,
        metric=metric,
        max_io_time=max_io_time
    )
    # Get score view
    score_view = filtered_view.reset_index()[[view_type, score_col, cut_col]]
    # Find filtered records and set duration scores
#     expanded_view = parent_view \
#         .query(f"{view_type}.isin(@indices)", local_dict={'indices': filtered_view.index.unique()}) \
#         .drop(columns=[score_col, cut_col], errors='ignore') \
#         .merge(score_view, on=[view_type])
    expanded_view = parent_view \
        .query(f"{view_type} in @indices", local_dict={'indices': filtered_view.index.unique()}) \
        .drop(columns=[score_col, cut_col], errors='ignore') \
        .merge(score_view, on=[view_type])
    # Set metric percentages
    expanded_view = _set_metric_percentages(ddf=expanded_view, max_io_time=max_io_time, metric=metric)
    # Return expanded view
    return expanded_view

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type]['expanded_view'] if parent_type in views else final_view
    # Create colum names
    metric_col, score_col, cut_col = f"{metric}_sum", f"{metric}_score", f"{metric}_cut"
    # Compute expanded view
    expanded_view = _compute_expanded_view(
        parent_view=parent_view,
        view_type=view_type,
        cols=(metric_col, score_col, cut_col),
        metric=metric,
        delta=0.0001,
        max_io_time=max_io_time
    )
    # Return views
    views[view_permutation] = {}
    views[view_permutation]['expanded_view'] = expanded_view
    views[view_permutation]['bottleneck_view'] = expanded_view[expanded_view[cut_col] >= 0.5]

len(views)

In [ ]:
for view_key, view_dict in views.items():
    print(view_key, 'expanded_view', len(view_dict['expanded_view']), 'bottleneck_view', len(view_dict['bottleneck_view']))

In [ ]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_id=('file_name', 'proc_name', 'trange'),
    proc_id=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_id='file_name',
    proc_id='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    ll_view: pd.DataFrame,
    ml_view: pd.DataFrame,
    hl_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = ll_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = hl_view.loc[hl_id]
        ml_row = ml_view.loc[(hl_id, ml_id)]
        ll_row = ll_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view_dict in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view_dict=view_dict,
            view_types=view_types,
        )
    # Generate bottlenecks
#     bottlenecks = _process_bottleneck_views(bottleneck_views=bottleneck_views)
    # Return bottleneck views
    return bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, pd.DataFrame]):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        bottlenecks_delayed.append(_process_bottleneck_view(
            view_key=view_key,
            ll_view=view_dict['low_level_view'],
            ml_view=view_dict['mid_level_view'],
            hl_view=view_dict['high_level_view']
        ))
    # Compute all bottlenecks
    futures = compute(*bottlenecks_delayed, sync=False)
    results = get_client().gather(list(futures))
    for view_key, result in results:
        bottlenecks[view_key] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
        view_key: tuple,
        view_dict: Dict[str, dd.DataFrame],
        view_types: list
    ):
        # Get view type
        view_type = view_key[-1]
        bottleneck_type = BOTTLENECK_TYPE[view_type]

        # Get parent view
        bottleneck_view = view_dict['bottleneck_view']

        # Create lower level view
        low_level_view = bottleneck_view \
            .groupby(list(BOTTLENECK_ORDER[view_type])) \
            .first() \
            .drop(columns=['acc_pat', 'io_cat'], errors='ignore')

        # Non-proc agg columns
        non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
        proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

        if bottleneck_type is not 'proc_name':

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'proc_name']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(proc_agg_dict)

        else:

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'trange']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(non_proc_agg_dict)

        return dict(
            low_level_view=low_level_view.persist(),
            mid_level_view=mid_level_view.persist(),
            high_level_view=high_level_view.persist()
        )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict['duration_cut'] = 'first'
    agg_dict['duration_score'] = 'first'
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


In [ ]:
bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottleneck_views)

In [ ]:
for view_key, bottleneck_view_dict in bottleneck_views.items():
    print(view_key, 'hl_view', len(bottleneck_view_dict['high_level_view']), 'ml_view', len(bottleneck_view_dict['mid_level_view']), 'll_view', len(bottleneck_view_dict['low_level_view']))

In [ ]:
%%timeit -r 10

trange_agg = ddf \
    .groupby(['trange']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()
trange_agg.sort_values(('duration', 'sum'), ascending=False)  

13.4 s ± 568 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 30s (started: 2023-01-11 15:28:26 -08:00)


In [ ]:
%%timeit -r 10

trange_agg2 = ddf \
    .groupby(['trange', 'io_cat']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()
trange_agg2.sort_values(('duration', 'sum'), ascending=False)  

14 s ± 292 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 34s (started: 2023-01-11 15:30:57 -08:00)


In [ ]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()

1min 26s ± 701 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 8min 41s (started: 2023-01-18 16:09:47 -08:00)


In [ ]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'size':[sum],
#         'size': [min, max, 'mean', sum],
#         'index':['count']
    }) \
    .compute()

1min 34s ± 660 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 9min 27s (started: 2023-01-18 16:18:29 -08:00)


In [ ]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'size':[sum],
#         'size': [min, max, 'mean', sum],
        'index':['count']
    }) \
    .compute()

1min 44s ± 757 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 10min 26s (started: 2023-01-18 16:43:30 -08:00)


In [ ]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
#         'size':[sum],
        'size': ['mean', sum],
        'index':['count']
    }) \
    .compute()

1min 54s ± 920 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 11min 25s (started: 2023-01-18 17:36:25 -08:00)


In [ ]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
#         'size':[sum],
        'size': [min, max, 'mean', sum],
        'index':['count']
    }) \
    .compute()

2min 14s ± 1.32 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 13min 23s (started: 2023-01-18 17:48:35 -08:00)


In [ ]:
%%timeit -r 10

ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

2min 28s ± 6.59 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 27min 7s (started: 2023-01-19 20:27:37 -08:00)


In [ ]:
%%timeit -r 10

ddf['acc_pat'] = ddf['acc_pat'].astype('i4')
ddf['io_cat'] = ddf['io_cat'].astype('i4')

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

3min 1s ± 6.43 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 32min 52s (started: 2023-01-19 21:03:58 -08:00)


In [ ]:
%%timeit -r 10

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat'], dropna=False, observed=True) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

2min 26s ± 2.13 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 26min 56s (started: 2023-01-19 13:33:38 -08:00)


In [ ]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i4')
ddf['io_cat'] = ddf['io_cat'].astype('i4')

all_agg = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

time: 3min 16s (started: 2023-01-19 21:37:16 -08:00)


In [ ]:
all_agg.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54655536 entries, (1, 113104469769545699, 109922137388615755, 3) to (2050, 113104469769545699, 7523492662002962052, 0)
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   (duration, sum)  float32
 1   (acc_pat, min)   int32  
 2   (acc_pat, max)   int32  
 3   (size, min)      int64  
 4   (size, max)      int64  
 5   (size, sum)      int64  
 6   (index, count)   int64  
dtypes: float32(1), int32(2), int64(4)
memory usage: 3.4 GB
time: 1.35 s (started: 2023-01-19 21:40:33 -08:00)


In [10]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

time: 2min 19s (started: 2023-01-25 22:16:32 -08:00)


In [10]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg_to_persist = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .reset_index() \
    .repartition('100MB')

all_agg_to_persist.columns = ['_'.join(tup).rstrip('_') for tup in all_agg_to_persist.columns.values]
all_agg_to_persist.to_parquet(f"{log_dir}/agg/")

[None]

time: 3min 52s (started: 2023-01-25 23:13:37 -08:00)


In [11]:
all_agg_from_persist = dd.read_parquet(f"{log_dir}/agg/*.parquet")
all_agg_from_persist

,trange,file_id,proc_id,io_cat,duration_sum,acc_pat_min,acc_pat_max,size_min,size_max,size_sum,index_count
npartitions=39,,,,,,,,,,,
0,int64,int64,int64,int64,float32,int8,int8,int64,int64,int64,int64
1401424,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
53254112,...,...,...,...,...,...,...,...,...,...,...
54655535,...,...,...,...,...,...,...,...,...,...,...


time: 895 ms (started: 2023-01-25 23:17:29 -08:00)


In [21]:
all_agg_to_persist = all_agg.copy()
all_agg_to_persist.columns = ['_'.join(tup).rstrip('_') for tup in all_agg_to_persist.columns.values]
# all_agg_to_persist.to_parquet(f"{log_dir}/agg")
all_agg_to_persist.columns

Index(['duration_sum', 'acc_pat_min', 'acc_pat_max', 'size_min', 'size_max',
       'size_sum', 'index_count'],
      dtype='object')

time: 368 ms (started: 2023-01-25 22:32:25 -08:00)


In [17]:
all_agg.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54655536 entries, (1, 113104469769545699, 109922137388615755, 3) to (2050, 113104469769545699, 7523492662002962052, 0)
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   (duration, sum)  float32
 1   (acc_pat, min)   int8   
 2   (acc_pat, max)   int8   
 3   (size, min)      int64  
 4   (size, max)      int64  
 5   (size, sum)      int64  
 6   (index, count)   int64  
dtypes: float32(1), int64(4), int8(2)
memory usage: 3.1 GB
time: 1.34 s (started: 2023-01-19 21:43:45 -08:00)


In [18]:
all_agg.groupby(level='trange').sum().sort_values(('duration', 'sum'), ascending=False)


duration acc_pat             size                          index
                sum     min   max        min        max         sum   count
trange                                                                     
109     1721.090942     0.0  40.0  693697375  758505618  3858948395  460578
111     1690.369751     0.0  39.0  768581010  822274763  5405174646  561200
108     1677.583862     0.0  38.0  694670290  730404524  4355503898  438780
107     1543.197266     0.0  37.0  589040865  617570274  4356702262  481463
113     1475.074829     0.0  36.0  546397583  605311855  5001712675  563823
...             ...     ...   ...        ...        ...         ...     ...
3087       0.000000     0.0   0.0          0          0           0       1
3084       0.000000     0.0   0.0          0          0           0       1
3080       0.000000     0.0   0.0          0          0           0       1
2839       0.000000     0.0   0.0          0          0           0       1
3107       0.000000     0.0   0.0          0          0           0       1

[2589 rows x 7 columns]

time: 3.62 s (started: 2023-01-19 21:44:11 -08:00)


In [27]:
all_agg.reset_index(['file_id', 'proc_id']).groupby(['trange', 'io_cat']).nunique()


file_id proc_id duration acc_pat     size                index
                                   sum     min max  min  max mean  sum count
trange io_cat                                                               
1      1           14     518      525       1   1    3    3    8  170   167
       2          114       1       13       1   1  106  106  106  106     1
       3          263    3870     3466       1   2    2    1    4    2   452
2      1           20     385      379       1   1    4    4    7  243   241
       2         1005       9      200       1   1  527  527  527  527     1
...               ...     ...      ...     ...  ..  ...  ...  ...  ...   ...
3254   3           88       1       88       1   1    1    1    1    1     1
3255   3           64       1       63       1   1    1    1    1    1     1
3256   3           76       1       76       1   1    1    1    1    1     1
3257   0            1       1        1       1   1    1    1    1    1     1
       3           24       1       24       1   1    1    1    1    1     1

[7718 rows x 10 columns]

time: 1min 7s (started: 2023-01-19 19:51:52 -08:00)


In [28]:
all_agg.reset_index(['trange', 'proc_id']).groupby(['file_id']).nunique()


trange proc_id duration acc_pat     size               \
                                         sum     min max  min max mean sum   
file_id                                                                      
-9223371986035283781      1       1        1       1   1    1   1    1   1   
-9223363969247508558      1       1        1       1   1    1   1    1   1   
-9223362727731233329      1       1        1       1   1    1   1    1   1   
-9223362290314498489      1       1        1       1   1    1   1    1   1   
-9223362230599577161      1       1        1       1   1    1   1    1   1   
...                     ...     ...      ...     ...  ..  ...  ..  ...  ..   
 9223361494309915981      1       1        1       1   1    1   1    1   1   
 9223362658809780944      1       1        1       1   1    1   1    1   1   
 9223364712007579008      1       1        1       1   1    1   1    1   1   
 9223368457304021926      1       1        1       1   1    1   1    1   1   
 9223369538921024184      1       1        1       1   1    1   1    1   1   

                     index  
                     count  
file_id                     
-9223371986035283781     1  
-9223363969247508558     1  
-9223362727731233329     1  
-9223362290314498489     1  
-9223362230599577161     1  
...                    ...  
 9223361494309915981     1  
 9223362658809780944     1  
 9223364712007579008     1  
 9223368457304021926     1  
 9223369538921024184     1  

[21267734 rows x 10 columns]

time: 1min 45s (started: 2023-01-19 19:59:05 -08:00)


In [17]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg_persisted = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .reset_index() \
    .persist()

time: 109 ms (started: 2023-01-24 21:35:03 -08:00)


In [18]:
all_agg_persisted

Dask DataFrame Structure:
              trange file_id proc_id io_cat duration acc_pat         size                index
                                                 sum     min   max    min    max    sum  count
npartitions=1                                                                                 
               int64   int64   int64  int64  float32    int8  int8  int64  int64  int64  int64
                 ...     ...     ...    ...      ...     ...   ...    ...    ...    ...    ...
Dask Name: reset_index, 1 tasks

time: 41.4 ms (started: 2023-01-24 21:35:05 -08:00)


In [19]:
trange_view_persisted = all_agg_persisted[['trange', 'duration']] \
    .groupby(['trange']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 36.7 ms (started: 2023-01-24 21:35:05 -08:00)


In [20]:
file_id_view_persisted = all_agg_persisted[['file_id', 'duration']] \
    .groupby(['file_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 35 ms (started: 2023-01-24 21:35:06 -08:00)


In [21]:
proc_id_view_persisted = all_agg_persisted[['proc_id', 'duration']] \
    .groupby(['proc_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 34 ms (started: 2023-01-24 21:35:07 -08:00)


In [22]:
wait([trange_view_persisted, file_id_view_persisted, proc_id_view_persisted])

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-035c237037e2f63fca4d036c72caac32', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-87c83f348b5e9330e998567f97ce4291', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-ae7723beeb60667f5ef62cde1bb109c8', 0)>}, not_done=set())

time: 2min 5s (started: 2023-01-24 21:35:08 -08:00)


In [24]:
proc_id_view_persisted.head()

,duration
,sum
proc_id,
8712563415903635531,1359.907715
6865659738399443019,1345.294189
6865660408414341195,1343.329102
5225694538768582731,1342.854126
8719823684455368779,1334.649414


time: 25.4 ms (started: 2023-01-24 21:37:51 -08:00)


In [26]:
trange_view_persisted.head()

,duration
,sum
trange,
109,1721.090942
111,1690.369751
108,1677.583862
107,1543.197266
113,1475.074829


time: 29.4 ms (started: 2023-01-24 21:39:01 -08:00)


In [27]:
trange_view_filtered = trange_view_persisted \
    .map_partitions(lambda df: df.assign(csp=df[('duration', 'sum')].cumsum() / df[('duration', 'sum')].sum())) \
    .map_partitions(lambda df: df.assign(delta=df['csp'].diff().fillna(df['csp'])))

trange_view_filtered = trange_view_filtered \
    .loc[trange_view_filtered['delta'] > DELTA] \
    .drop(columns=['csp', 'delta']) \
    .persist()

trange_view_filtered

,duration
,sum
npartitions=1,
,float32
,...


time: 65.6 ms (started: 2023-01-24 21:40:00 -08:00)


In [29]:
trange_view_filtered.head()

,duration
,sum
trange,
109,1721.090942
111,1690.369751
108,1677.583862
107,1543.197266
113,1475.074829


time: 30.6 ms (started: 2023-01-24 21:40:10 -08:00)


In [30]:
all_agg_persisted.index

Dask Index Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: reset_index, 2 tasks

time: 7.68 ms (started: 2023-01-24 21:40:16 -08:00)


In [31]:
trange_view_indices = list(set(trange_view_filtered.index.compute()))
trange_llc = all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)] \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sum() \
    .persist()
#     .sort_values(('duration', 'sum'), ascending=False) \
#     .persist()

time: 77.6 ms (started: 2023-01-24 21:41:38 -08:00)


In [32]:
trange_llc

Dask DataFrame Structure:
              duration acc_pat         size                index
                   sum     min   max    min    max    sum  count
npartitions=1                                                   
               float32    int8  int8  int64  int64  int64  int64
                   ...     ...   ...    ...    ...    ...    ...
Dask Name: dataframe-groupby-sum-agg, 1 tasks

time: 33.9 ms (started: 2023-01-24 21:41:41 -08:00)


In [33]:
wait(trange_llc)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-1dcf0f630d65d8e187e5b8b674fbb1ca', 0)>}, not_done=set())

time: 26.4 s (started: 2023-01-24 21:41:48 -08:00)


In [36]:
trange_llc.count().compute()

duration  sum      25059324
acc_pat   min      25059324
          max      25059324
size      min      25059324
          max      25059324
          sum      25059324
index     count    25059324
dtype: int64

time: 489 ms (started: 2023-01-24 21:42:37 -08:00)


In [39]:
ddf.index.count().compute()

715248240

time: 12.8 s (started: 2023-01-24 21:45:31 -08:00)


In [40]:
25059324 / 715248240

0.03503584154223155

time: 3.74 ms (started: 2023-01-24 21:46:43 -08:00)


In [ ]:
%%timeit -r 10

trange_agg = ddf \
    .groupby(['trange', 'io_cat']) \
    .agg({'duration':[sum]}) \
    .compute() \
    .sort_values(('duration', 'sum'), ascending=False)

In [33]:
trange_agg['duration', 'cumsum'] = trange_agg['duration', 'sum'].cumsum() / trange_agg['duration', 'sum'].sum()
# trange_agg_file['duration', 'delta'] = trange_agg_file['duration', 'cumsum'] / trange_agg_file['duration', 'cumsum'].shift()
trange_agg['duration', 'delta'] = trange_agg['duration', 'cumsum'].diff().fillna(trange_agg['duration', 'cumsum'])
trange_agg['duration', 'per'] = trange_agg.div(trange_agg['duration', 'sum'].sum(), level=0)['duration', 'sum'] * 100
trange_agg_cut =  trange_agg[trange_agg['duration', 'delta'] > DELTA]
trange_agg_cut = trange_agg_cut.reindex(sorted(trange_agg_cut.columns), axis=1)
trange_agg_cut

duration                                    index
                 cumsum     delta       per         sum    count
trange io_cat                                                   
1      3       0.001178  0.001178  0.117756  157.112274  4738612
2      3       0.003216  0.002039  0.203882  272.024048  2187539
3      1       0.004104  0.000888  0.088809  118.490753  9399964
       3       0.006605  0.002500  0.250014  333.574982  1892752
4      3       0.009288  0.002684  0.268370  358.065979  1428528
...                 ...       ...       ...         ...      ...
82     1       0.979731  0.000144  0.014394   19.205080  1840564
83     1       0.979846  0.000115  0.011461   15.291310  1500505
95     1       0.980222  0.000196  0.019592   26.140547  2055562
96     1       0.980393  0.000171  0.017152   22.884914  1739375
1331   3       0.980979  0.000117  0.011681   15.584929    19290

[804 rows x 5 columns]

time: 36.4 ms (started: 2023-01-10 21:03:27 -08:00)


In [43]:
trange_agg_cut_piv = trange_agg_cut.reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,read,write,metadata
trange,,,
1,39.384121,0.0,157.112274
2,102.217094,0.0,272.024048
3,118.490753,0.0,333.574982
4,128.453979,0.0,358.065979
5,136.352509,0.0,389.777374
...,...,...,...
1106,0.000000,0.0,15.551300
1115,0.000000,0.0,16.820549
1138,0.000000,0.0,14.048192


time: 33.1 ms (started: 2023-01-10 21:07:36 -08:00)


In [31]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10_3

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

p = figure(width=800, height=200)
p.vbar_stack(impacts, width=0.7, x="trange", source=source, color=Category10_3, legend_label=impacts)

p.y_range.start = 0
p.xgrid.grid_line_color = None
p.legend.location = "top_right"

show(p)

time: 338 ms (started: 2023-01-10 21:00:51 -08:00)


In [39]:
ddf = ddf.persist()

time: 51.7 ms (started: 2023-01-03 00:19:35 -08:00)


In [40]:
hlm_trange = compute_hlm(ddf=ddf, group_by='trange')
hlm_trange

,duration
,sum
npartitions=1,
,float32
,...


time: 101 ms (started: 2023-01-03 00:19:44 -08:00)


In [41]:
hlm_trange.compute()

,duration
,sum
trange,
"(1080000000.0, 1090000000.0]",1721.090942
"(1100000000.0, 1110000000.0]",1690.369751
"(1070000000.0, 1080000000.0]",1677.583862
"(1060000000.0, 1070000000.0]",1543.197388
"(1120000000.0, 1130000000.0]",1475.074829
...,...
"(27410000000.0, 27420000000.0]",0.000000
"(27420000000.0, 27430000000.0]",0.000000


time: 3.9 s (started: 2023-01-03 00:19:45 -08:00)


In [43]:
hlm_trange_filtered = filter_delta_delayed(df=hlm_trange, delta=DELTA)
hlm_trange_filtered.compute()

duration                       
                                     csp     delta          sum
trange                                                         
(1080000000.0, 1090000000.0]    0.012900  0.012900  1721.090942
(1100000000.0, 1110000000.0]    0.025569  0.012669  1690.369751
(1070000000.0, 1080000000.0]    0.038142  0.012574  1677.583862
(1060000000.0, 1070000000.0]    0.049709  0.011566  1543.197388
(1120000000.0, 1130000000.0]    0.060764  0.011056  1475.074829
...                                  ...       ...          ...
(6580000000.0, 6590000000.0]    0.943540  0.000101    13.457269
(5700000000.0, 5710000000.0]    0.943641  0.000101    13.429598
(11020000000.0, 11030000000.0]  0.943742  0.000101    13.426023
(4130000000.0, 4140000000.0]    0.943842  0.000101    13.419438
(3950000000.0, 3960000000.0]    0.943942  0.000100    13.392311

[626 rows x 3 columns]

time: 3.47 s (started: 2023-01-03 00:20:26 -08:00)


In [ ]:
hlm_trange_filtered_file_id = compute_hlm2(ddf=ddf[ddf['trange'].isin(hlm_trange_filtered.index)], group_by='file_id')
hlm_trange_filtered_file_id

In [10]:
stat_agg = ddf \
    .groupby(['io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
stat_agg = stat_agg[stat_agg.index.isin([1, 2, 3])]
stat_agg['duration', 'per'] = stat_agg.div(stat_agg['duration', 'sum'].sum(), level=0)['duration', 'sum']
stat_agg['file_id', 'per'] = stat_agg.div(stat_agg['file_id', 'nunique'].max(), level=0)['file_id', 'nunique']
stat_agg['index', 'per'] = stat_agg.div(stat_agg['index', 'count'].sum(), level=0)['index', 'count']
stat_agg['proc_id', 'per'] = stat_agg.div(stat_agg['proc_id', 'nunique'].max(), level=0)['proc_id', 'nunique']
stat_agg['size', 'per'] = stat_agg.div(stat_agg['size', 'sum'].sum(), level=0)['size', 'sum']
stat_agg['bw', 'sum'] = stat_agg['size', 'sum'] / stat_agg['duration', 'sum']
stat_agg['bw', 'per'] = stat_agg.div(stat_agg['bw', 'sum'].sum(), level=0)['bw', 'sum']
stat_agg['xfer', 'max'] = pd.cut(stat_agg['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'min'] = pd.cut(stat_agg['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'mean'] = pd.cut(stat_agg['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['bw', 'sum'] = stat_agg['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
stat_agg['size', 'sum'] = stat_agg['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
stat_agg.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
stat_agg = stat_agg.reindex(sorted(stat_agg.columns), axis=1)
stat_agg


acc_pat         app            bw             duration                 \
           max min nunique           per        sum       per            sum   
io_cat                                                                         
1            0   0       5  9.967778e-01   9.28GB/s  0.058672    7828.180176   
2            0   0       5  3.222178e-03   0.03GB/s  0.026136    3487.186035   
3            1   0       7 -3.179351e-11  -0.00GB/s  0.915191  122106.781250   

         file_id           hostname      index           proc_id            \
         nunique       per  nunique      count       per nunique       per   
io_cat                                                                       
1       10549343  0.496026       32  526812254  0.736567    1764  0.442105   
2       10445412  0.491139       32   10730937  0.015004    1764  0.442105   
3       21267734  1.000000       32  177682895  0.248429    3990  1.000000   

          rank          size               xfer                
       nunique           per         sum    max    mean   min  
io_cat                                                         
1            1  9.985621e-01  72644.43GB  ~16MB  ~256KB  <4KB  
2            1  1.437939e-03    104.61GB  ~16MB   ~16KB  <4KB  
3         1280 -4.968143e-10     -0.00GB   <4KB    <4KB  <4KB

time: 1min 1s (started: 2023-01-11 12:53:21 -08:00)


In [11]:
# importing the modules
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.transform import cumsum
import math

data = stat_agg.reset_index()
data['io_cat'] = ['read', 'write', 'metadata']
data['duration', 'angle'] = data['duration', 'per'] * 2*math.pi
data['duration', 'color'] = Category10[3]
data['duration', 'per_fmt'] = data['duration', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['size', 'angle'] = data['size', 'per'] * 2*math.pi
data['size', 'color'] = Category10[3]
data['size', 'per_fmt'] = data['size', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['index', 'angle'] = data['index', 'per'] * 2*math.pi
data['index', 'color'] = Category10[3]
data['index', 'per_fmt'] = data['index', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")

data

io_cat acc_pat         app            bw             duration  \
                max min nunique           per        sum       per   
0      read       0   0       5  9.967778e-01   9.28GB/s  0.058672   
1     write       0   0       5  3.222178e-03   0.03GB/s  0.026136   
2  metadata       1   0       7 -3.179351e-11  -0.00GB/s  0.915191   

                   file_id            ...  xfer  duration                   \
             sum   nunique       per  ...   min     angle    color per_fmt   
0    7828.180176  10549343  0.496026  ...  <4KB  0.368649  #1f77b4   5.87%   
1    3487.186035  10445412  0.491139  ...  <4KB  0.164220  #ff7f0e   2.61%   
2  122106.781250  21267734  1.000000  ...  <4KB  5.750317  #2ca02c  91.52%   

           size                      index                   
          angle    color per_fmt     angle    color per_fmt  
0  6.274150e+00  #1f77b4  99.86%  4.627990  #1f77b4  73.66%  
1  9.034839e-03  #ff7f0e   0.14%  0.094270  #ff7f0e   1.50%  
2 -3.121577e-09  #2ca02c  -0.00%  1.560925  #2ca02c  24.84%  

[3 rows x 30 columns]

time: 55.6 ms (started: 2023-01-11 12:54:22 -08:00)


In [12]:
from bokeh.layouts import column, row

def pie_chart(by: str):
        p = figure(height=200, width=300, title=f"{by} by I/O category", toolbar_location=None, 
                tooltips=f"@io_cat_: @{by}_per_fmt (x)", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=0, radius=0.4,
                start_angle=cumsum(f'{by}_angle', include_zero=True), end_angle=cumsum(f'{by}_angle'),
                fill_color=f'{by}_color', line_color=None, legend_field='io_cat_', source=data)

        p.axis.axis_label = None
        p.axis.visible = False
        p.grid.grid_line_color = None

        return p

p_dur = pie_chart('duration')
p_ix = pie_chart('index')
p_size = pie_chart('size')

show(row([p_dur, p_ix, p_size]))

time: 354 ms (started: 2023-01-11 12:54:22 -08:00)
